Initial Assessment

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
df = pd.read_csv('openipf-latest/openipf-2023-08-05/openipf-2023-08-05-ae3dc469.csv', na_filter=True)
df.shape

C:\Users\Ananya\AppData\Local\Temp\ipykernel_20304\2425286704.py:1: DtypeWarning: Columns (33,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('openipf-latest/openipf-2023-08-05/openipf-2023-08-05-ae3dc469.csv', na_filter=True)


(1188859, 41)

In [11]:
df.head()

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
0,Karen Thomson,F,SBD,Single-ply,39.5,35-39,40-49,Masters 1,51.90,52,...,Yes,New Zealand,NaN,OceaniaPF,IPF,2002-12-06,New Zealand,BOP,Tauranga,Oceania Championships
1,Deborah Groves,F,SBD,Single-ply,30.5,24-34,24-39,Open,50.64,52,...,Yes,Australia,NaN,OceaniaPF,IPF,2002-12-06,New Zealand,BOP,Tauranga,Oceania Championships
2,Renee Orbon,F,SBD,Single-ply,31.5,24-34,24-39,Open,55.76,56,...,Yes,New Zealand,NaN,OceaniaPF,IPF,2002-12-06,New Zealand,BOP,Tauranga,Oceania Championships
3,Rikki Flanders,F,SBD,Single-ply,31.5,24-34,24-39,Open,56.42,60,...,Yes,Australia,NaN,OceaniaPF,IPF,2002-12-06,New Zealand,BOP,Tauranga,Oceania Championships
4,Julie Hallmond,F,SBD,Single-ply,NaN,NaN,50-59,Masters 2,58.20,60,...,Yes,New Zealand,NaN,OceaniaPF,IPF,2002-12-06,New Zealand,BOP,Tauranga,Oceania Championships


This is a big dataset. For tractability and modeling, it would be ideal to decrease it's size. I will investigate the contents of each column to see what is extraneous.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1188859 entries, 0 to 1188858
Data columns (total 41 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Name              1188859 non-null  object 
 1   Sex               1188859 non-null  object 
 2   Event             1188859 non-null  object 
 3   Equipment         1188859 non-null  object 
 4   Age               859909 non-null   float64
 5   AgeClass          902112 non-null   object 
 6   BirthYearClass    916781 non-null   object 
 7   Division          1187914 non-null  object 
 8   BodyweightKg      1174873 non-null  float64
 9   WeightClassKg     1176598 non-null  object 
 10  Squat1Kg          461065 non-null   float64
 11  Squat2Kg          456422 non-null   float64
 12  Squat3Kg          444892 non-null   float64
 13  Squat4Kg          327 non-null      float64
 14  Best3SquatKg      853454 non-null   float64
 15  Bench1Kg          657138 non-null   float64
 16  

In [8]:
df['AgeClass'].unique()

array(['35-39', '24-34', nan, '45-49', '50-54', '40-44', '18-19', '13-15',
       '16-17', '20-23', '55-59', '70-74', '60-64', '65-69', '75-79',
       '5-12', '80-999'], dtype=object)

In [9]:
df['BirthYearClass'].unique()

array(['40-49', '24-39', '50-59', nan, '60-69', '14-18', '19-23',
       '70-999'], dtype=object)

In IPF, Birth Year Class determines the age division one competes in. These correspond to Sub-junior, Junior, Open, Masters 1, 2, 3 classes. Age Class, however, does not align with these competition classes. The data dictionary says that Age Class is sometimes used by a federation without further providing age information. Information about age should be extracted from the Age Class column. I will use Age Class to impute information about age. Given the fact that the Age column is already an approximation (according to the data dictionary, for ages listed with a decimal), I believe that this imputation will not significantly affect the underlying distribution.

In [13]:
df['Division'].unique()

array(['Masters 1', 'Open', 'Masters 2', ..., 'Middle', 'Super',
       'SuperHeavy'], dtype=object)

Division seems to contain information from age classes and weight classes with no standardization. The data dictionary reads that this is free-form text to provide context for a meet because some federations have been pre-configured in the database. Since this information is free-form and can be inferred from other columns, I will drop it.

In [14]:
df['BodyweightKg'].head()

0    51.90
1    50.64
2    55.76
3    56.42
4    58.20
Name: BodyweightKg, dtype: float64

In [15]:
df['WeightClassKg'].unique()

array(['52', '56', '60', '67.5', '75', '82.5', '90', '90+', '100', '110',
       '125', '125+', nan, '48', '57', '63', '72', '84', '84+', '59',
       '66', '74', '83', '93', '105', '120', '120+', '47', '53', '44',
       '69', '43', '76', '63+', '110+', '105+', '30', '35', '40', '60+',
       '140+', '140', '82.5+', '75+', '57+', '72+', '69+', '93+', '76+',
       '100+', '67.5+', '50', '65', '80', '80+', '83+', '74+', '52+',
       '46', '49', '36', '57.1', '63.5', '69.8', '76.2', '88.9', '95.2',
       '101.6+', '62.5', '70', '89', '95.5', '101.5', '115', '68', '55.5',
       '58.5', '144.5', '144.5+', '145', '145+', '79.5', '90.5', '91+',
       '90.7+', '90.7', '215', '54.4', '74.8', '83.9', '92.9', '92.9+',
       '57.5', '68.5', '72.5', '77', '81.5', '87.5', '95', '92', '94',
       '70.7', '78.4', '58.9', '81.6', '104.3', '117.9', '117.9+', '84.8',
       '104.4', '82.4', '135.9', '65.3', '77.7', '84.1', '80.2', '103.4',
       '98.1', '86.1', '98.8', '146.6', '73.7', '107', '8

IPF Weight classes have changed over time. Instead of using every single weight class present, I will use the bodyweight column to create a new weight class column that corresponds to weight classes at the time of performing this EDA. This will change the meanings of some records (if records are noted here), but it will put past lifts into current weight contexts. Simply, the ordinality of best attempts will change, but the cardinality remains the same.

All attempts are optionally recorded, so there are plenty of missing values across all of the columns. 4th attempts are only used in some cases where the weight was misloaded or for single lift events (usually bench). I will preserve all of that for now and move onto other rows that can be dealt with more easily.

In [5]:
df['Federation'].describe()

count     1188859
unique        100
top         USAPL
freq       211453
Name: Federation, dtype: object

In [6]:
df['ParentFederation'].describe()

count     1188859
unique          1
top           IPF
freq      1188859
Name: ParentFederation, dtype: object

As expected, all parent federation values are IPF since this is the IPF specific dataset. I will simply drop this column.

In [17]:
df['Place'].unique()

array(['1', '2', '4', '3', 'G', 'DQ', '5', 'NS', '7', '6', '10', '9', '8',
       '11', 'DD', '12', '13', '14', '15', '16', '17', '18', '19', '20',
       '21', '22', '24', '23', '34', '25', '31', '28', '33', '27', '35',
       '26', '36', '30', '32', '29', '37', '38', '39', '40', '41', '42',
       '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53',
       '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64',
       '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75',
       '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86',
       '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97',
       '98', '99', '100', '101', '102', '103', '104', '105', '106', '107',
       '108', '109', '110', '111', '112', '113', '114', '115', '116',
       '117', '118', '119', '120', '195'], dtype=object)

This column includes ordinal placing at a meet and categories like disqualifications and no-shows. 

In order to clean this further and perform faster analysis given the size of the CSV, I will drop Division and ParentFederation, fill blanks as null, and then import it into SQL.

In [16]:
df.drop(columns=['Division', 'ParentFederation'], inplace=True)
df.fillna(value='NULL', inplace=True)

In [17]:
df.head()

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,BodyweightKg,WeightClassKg,Squat1Kg,...,Goodlift,Tested,Country,State,Federation,Date,MeetCountry,MeetState,MeetTown,MeetName
0,Karen Thomson,F,SBD,Single-ply,39.5,35-39,40-49,51.9,52,105.0,...,63.73,Yes,New Zealand,NULL,OceaniaPF,2002-12-06,New Zealand,BOP,Tauranga,Oceania Championships
1,Deborah Groves,F,SBD,Single-ply,30.5,24-34,24-39,50.64,52,122.5,...,76.76,Yes,Australia,NULL,OceaniaPF,2002-12-06,New Zealand,BOP,Tauranga,Oceania Championships
2,Renee Orbon,F,SBD,Single-ply,31.5,24-34,24-39,55.76,56,105.5,...,66.09,Yes,New Zealand,NULL,OceaniaPF,2002-12-06,New Zealand,BOP,Tauranga,Oceania Championships
3,Rikki Flanders,F,SBD,Single-ply,31.5,24-34,24-39,56.42,60,110.0,...,61.74,Yes,Australia,NULL,OceaniaPF,2002-12-06,New Zealand,BOP,Tauranga,Oceania Championships
4,Julie Hallmond,F,SBD,Single-ply,NULL,NULL,50-59,58.2,60,105.0,...,59.79,Yes,New Zealand,NULL,OceaniaPF,2002-12-06,New Zealand,BOP,Tauranga,Oceania Championships


In [18]:
df.to_csv('ipfdropped.csv', index=False)

SQL gaveme an error and I want to investigate what is happening in rows 20 - 24 to see what the problem is.

In [21]:
df['WeightClassKg'].iloc[20:24]

20      90
21     90+
22     90+
23    82.5
Name: WeightClassKg, dtype: object

It turns out that the "+" symbol makes these numbers not a number. I will change the SQL schema for this column to be a varchar and modify it in SQL later.